In [3]:
import csv
import pprint
import json
from datetime import datetime, timezone
import random

Get data

In [9]:
with open(".\\raw_data\\verified-2019_tweets.json", "r") as f:
    raw_data = json.load(f)

Fn to get account age

In [10]:
def get_age(account):

    """
    Finds
    """
    date_format = "%a %b %d %H:%M:%S %z %Y"

    created = datetime.strptime(account['created_at'], date_format)
    now = datetime.now(timezone.utc)

    delta = now - created

    return delta.days


Fn to get a line of a csv from an account

In [11]:
def get_line(account):
    name = account['name']
    screen_name = account['screen_name']
    desc = account ['description']
    followers = account['followers_count']
    following = account['friends_count']
    verified = account['verified']
    age = get_age(account)
    tweet_per_day = round(account['statuses_count']/age, 2)
    default_image = account['default_profile_image']
    
    return ",".join([name, screen_name, desc, str(followers), str(following), str(verified), str(age), str(tweet_per_day), str(default_image)])

Pick 100 random indices

In [12]:
sample = random.sample(range(len(raw_data)), 100)

Write the 100 random samples into a csv

In [13]:
with open('real_users_100.csv', "w", encoding="utf-8") as f:
    for number in sample:
        f.write(get_line(raw_data[number]['user']) + "\n")

In [14]:
def write_sql(account):
    
    name = account['name'].replace("'", "\\'").replace('"', '\"')
    screen_name = account['screen_name'].replace("'", "\\'").replace('"', '\"')
    desc = account ['description'].replace("\n", "\\n").replace("'", "\\'").replace('"', '\"')
    followers = account['followers_count']
    following = account['friends_count']
    verified = account['verified']
    age = get_age(account)
    tweet_per_day = round(account['statuses_count']/age, 2)
    default_image = account['default_profile_image']
    
    ret = "insert into twitter_users (display_name, acct_name, description, followers, following, verified, age, tweets_per_day, default_image, real_acct) values ("
    ret += f"'{name}','{screen_name}','{desc}',{followers},{following},'{verified}',{age},{tweet_per_day},'{default_image}','True');"
    
    return ret

In [15]:
with open('real_users_100.sql', "w", encoding="utf-8") as f:
    for number in sample:
        f.write(write_sql(raw_data[number]['user']) + "\n")